# Titnic

## 1. Prepare dataset

In [2]:
!kaggle competitions download -c titanic -p ./data/
import zipfile
with zipfile.ZipFile("./data/titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
